In [1]:
import polars as pl
import scipy.stats as stats
import polars.selectors as cs

In [15]:
def load_data():
    return (
    pl.read_excel('../data/splunk_projections.xlsx', sheet_name="The List")
    .select(
        pl.col('NAME').alias('player'),
        pl.col('RK').alias('rk'),
        pl.col('POS').alias('pos'),
        pl.col('TEAM').alias('team'),
        pl.col('FP').alias('pts'),
        pl.col('VORP').alias('vorp'),
        pl.col('ADP').fill_null(pl.max('ADP')).alias('adp'),
    )
    .with_columns(
        pl.col('adp').mul(2).add(pl.col('rk')).truediv(3).alias('tru')
    )
    .with_columns(
        (abs(pl.col('adp') - pl.col('tru'))).clip(0, 50).alias('adp_diff')
    )
    .with_columns(
        (0.15 * pl.col('tru') + 0.15 * pl.col('adp_diff')).alias('sd')
    )
    .select('player', 'rk', 'pos', 'team', 'pts', 'vorp', pl.col('tru').alias('adp'), pl.col('sd').alias('adp_sd'))
)

In [16]:
def process_probs(df):
    return (
    df
    .with_columns(
        pl.struct(['adp', 'adp_sd', pl.lit(pick_1).alias('pick')])
        .map_elements(availability_probability, return_dtype=pl.Float64)
        .alias("prob_1"),
        pl.struct(['adp', 'adp_sd', pl.lit(pick_2).alias('pick')])
        .map_elements(availability_probability, return_dtype=pl.Float64)
        .alias("prob_2"),
        pl.struct(['adp', 'adp_sd', pl.lit(pick_3).alias('pick')])
        .map_elements(availability_probability, return_dtype=pl.Float64)
        .alias("prob_3"),
    )
)

In [53]:
def process_dropoffs(df):
    return (
        df
        .with_columns([
        (pl.col('vorp') - get_next_round_expectation(df, pos, pick)).alias(f"dropoff_{pos}_{pick}")
        for pos in ['C', 'LW', 'RW', 'D', 'G']
        for pick in [1, 2, 3]]
    )
    .with_columns([
        pl.when(pl.col('pos').str.contains(pos)).then(pl.col(f"dropoff_{pos}_{pick}")).otherwise(pl.lit(0)).alias(
            f"dropoff_{pos}_{pick}")
        for pos in ['C', 'LW', 'RW', 'D', 'G']
        for pick in [1, 2, 3]]
    )
    .with_columns([
        pl.max_horizontal([cs.contains(f'_{pick}').and_(cs.contains('dropoff'))]).alias(f'dropoff_{pick}') for pick in
        [1, 2, 3]]
    )
    .select('player','pos','vorp',pl.col('prob_1').round(2),pl.col('prob_2').round(2),'dropoff_1','dropoff_2')
    .with_columns(scaled=pl.col('vorp').add(pl.col('dropoff_1')).add(pl.col('dropoff_2')))
    )

In [54]:
def get_pick_numbers(num_teams_in_draft, my_pick_slot, num_rounds):
    """
    For a serpentine draft, return the pick numbers for the team that starts at `my_pick_slot`.

    Args:
        num_teams_in_draft (int): The total number of teams in the draft.
        my_pick_slot (int): Your pick position in the first round (1-indexed).
        num_rounds (int): The total number of rounds in the draft.

    Returns:
        list: A list of pick numbers for each round.
    """
    picks = []

    for round_num in range(1, num_rounds + 1):
        if round_num % 2 != 0:
            # Odd rounds: normal order
            pick_number = (round_num - 1) * num_teams_in_draft + my_pick_slot
        else:
            # Even rounds: reverse order
            pick_number = round_num * num_teams_in_draft - (my_pick_slot - 1)

        picks.append(pick_number)

    return picks

In [55]:
def availability_probability(row):
    adp, adp_sd, pick = row['adp'], row['adp_sd'], row['pick']
    if adp_sd == 0:  # Prevent division by zero for players with no variance
        return 1.0 if pick < adp else 0.0
    return 1 - stats.norm.cdf(pick, loc=adp, scale=adp_sd)

In [56]:
def get_next_round_expectation(df, position, pick_number):
    return (
        df
        .filter(pl.col('pos').str.contains(position))
        .with_columns(
            (1 - pl.col(f"prob_{pick_number}")).alias("prob_drafted")  # Probability the player is drafted
        )
        .with_columns(
            (pl.col("prob_drafted").cum_prod().shift(1, fill_value=1)).alias("prob_all_above_drafted")
        )
        .with_columns(
            (pl.col("prob_all_above_drafted") * pl.col(f"prob_{pick_number}")).alias("prob_best_available")
        )
        .with_columns(
            (pl.col('vorp') * pl.col('prob_best_available')).alias('vorp_weighted')
        )
        .select(pl.sum('vorp_weighted')).item()
    )

In [57]:
all_picks = get_pick_numbers(14, 1, 20)

In [58]:
current_pick = 1
pick_1 = [pick for pick in all_picks if pick > current_pick][0]
pick_2 = [pick for pick in all_picks if pick > pick_1][0]
pick_3 = [pick for pick in all_picks if pick > pick_2][0]

In [59]:
(
    load_data()
    .pipe(process_probs)
    .pipe(process_dropoffs)
    .head(50)
    .to_pandas()
)

,player,pos,vorp,prob_1,prob_2,dropoff_1,dropoff_2,scaled
0,Connor McDavid,C,435.107631,0.00,0.00,309.386651,310.451239,1054.945521
1,Nathan MacKinnon,C,431.860310,0.00,0.00,306.139331,307.203918,1045.203559
2,Auston Matthews,C,414.813771,0.00,0.00,289.092791,290.157379,994.063941
3,David Pastrnak,RW,339.758330,0.00,0.00,191.367836,197.251595,728.377760
4,Nikita Kucherov,RW,332.064444,0.00,0.00,183.673950,189.557709,705.296103
5,Cale Makar,D,278.781161,0.00,0.00,108.096362,110.806209,497.683732
6,Mikko Rantanen,RW,269.318980,0.00,0.00,120.928486,126.812245,517.059711
7,Leon Draisaitl,C/LW,266.303835,0.00,0.00,140.582855,141.647443,548.534133
8,Igor Shesterkin,G,258.386584,0.00,0.00,59.135089,63.200105,380.721778
9,Roman Josi,D,256.455353,0.00,0.00,85.770553,88.480401,430.706307
